# 521153S Deep Learning Final Project

In [1]:
# import necessary packages
import os
import requests
import zipfile
import sys
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torchvision
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, datasets
from torchvision.io import read_image
from torchvision.models import resnet18, resnet34, resnet50, resnet101, resnet152, ResNet18_Weights, ResNet34_Weights, ResNet50_Weights, ResNet101_Weights, ResNet152_Weights, vgg11, vgg13, vgg16, vgg19, VGG11_Weights, VGG13_Weights, VGG16_Weights, VGG19_Weights
import gdown
import urllib
import os
import random
import tarfile
import time
import shutil
import logging as lg
from tqdm.notebook import tqdm
from torch.utils.data import random_split
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset

In [2]:
# Set the logging level
# Possible Levels: DEBUG, INFO,
lg.basicConfig(level=lg.DEBUG)

In [3]:
# download the dataset
val_url = 'https://drive.google.com/u/0/uc?id=1hSMUMj5IRpf-nQs1OwgiQLmGZCN0KDWl'
train_url = 'https://drive.google.com/u/0/uc?id=107FTosYIeBn5QbynR46YG91nHcJ70whs'
test_url = 'https://drive.google.com/u/0/uc?id=1yKyKgxcnGMIAnA_6Vr2ilbpHMc9COg-v'
eurosat_url = 'https://zenodo.org/records/7711810/files/EuroSAT_RGB.zip?download=1'

val_file = './data/val.tar'
train_file = './data/train.tar'
test_file = './data/test.tar'
eurosat_file = './data/eurosatrgb.zip'


if not os.path.exists('./data'):
    print('Creating data directory')
    os.mkdir('./data')

if not os.path.exists('./data/val.tar'):
    print('Downloading val.tar')
    gdown.download(val_url, val_file)
    val_tar = tarfile.open(val_file)
    val_tar.extractall('./data/')
    val_tar.close()

if not os.path.exists(train_file):
    print('Downloading train.tar')
    gdown.download(train_url, train_file)
    train_tar = tarfile.open(train_file)
    train_tar.extractall('./data/')
    train_tar.close()

if not os.path.exists(test_file):
    print('Downloading test.tar')
    gdown.download(test_url, test_file)
    test_tar = tarfile.open(test_file)
    test_tar.extractall('./data/')
    test_tar.close()

if not os.path.exists(eurosat_file):
    print('Downloading EuroSAT_RGB.zip')
    response = urllib.request.urlretrieve(eurosat_url, eurosat_file)
    eurosat_zip = zipfile.ZipFile(eurosat_file)
    eurosat_zip.extractall('./data/eurosat')
    eurosat_zip.close()

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# If you encounter some issues regarding cuda device, e.g., "RuntimeError: CUDA Out of memory error",
# try to switch the device to cpu by using the following code

# device = torch.device('cpu')
print('Device:', device)

Device: cuda:0


In [5]:
# Image Size
image_size = 224

# Batch size during training
batch_size = 128
num_workers = 1

# Learning rate for optimizers (id not good, decrease learning rate (try 0.001, 0.0001, 0.00001))
lr = 0.001

# Momentum
momentum = 0.9

# Number of training epochs
num_epochs = 10

# Weight decay (if not good, increase weight_decay (try 1e-4, 1e-3, 1e-2))
weight_decay=1e-2

In [6]:
model_resnet18 = resnet18(weights=ResNet18_Weights.DEFAULT)
model_resnet34 = resnet34(weights=ResNet34_Weights.DEFAULT)
model_resnet50 = resnet50(weights=ResNet50_Weights.DEFAULT)
model_resnet101 = resnet101(weights=ResNet101_Weights.DEFAULT)
model_resnet152 = resnet152(weights=ResNet152_Weights.DEFAULT)

model_vgg11 = vgg11(weights=VGG11_Weights.DEFAULT)
model_vgg13 = vgg13(weights=VGG13_Weights.DEFAULT)
model_vgg16 = vgg16(weights=VGG16_Weights.DEFAULT)
model_vgg19 = vgg19(weights=VGG19_Weights.DEFAULT)


In [ ]:
"""
train_dataset = torchvision.datasets.ImageFolder(
    root='./data/train',
    transform=transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]), # mean and std for ImageNet dataset
    ])
)


test_dataset = torchvision.datasets.ImageFolder(
    root='./data/test',
    transform=transforms.Compose([
        #transforms.Resize(image_size),
        #transforms.CenterCrop(image_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])
)

validation_dataset = torchvision.datasets.ImageFolder(
    root='./data/val',
    transform=transforms.Compose([
        #transforms.Resize(image_size),
        #transforms.CenterCrop(image_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])
)


dataloader_train = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
dataloader_test = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
dataloader_validation = torch.utils.data.DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
"""

In [7]:
# Stratified split, tries to maintain a proportional distribution of samples for each class in all three sets (train, val and test).

# Val and Test ratio total, Train ratio (1-test_ratio)
test_ratio = 0.3

# Split between Val and Test
test_val_split = 0.5

# Loading the ImageFolder dataset
train_dataset = torchvision.datasets.ImageFolder(root='./data/train')

# Getting the number of classes
num_classes = len(train_dataset.classes)

# Initializing lists to store indices for train, validation, and test sets
train_indices, val_indices, test_indices = [], [], []

# Splitting each class separately
for class_index in range(num_classes):
    # Getting indices for samples belonging to the current class
    class_indices = np.where(np.array(train_dataset.targets) == class_index)[0]
    
    # Splitting indices for each class separately into train, validation, and test sets
    train_idx, temp_idx = train_test_split(class_indices, test_size=test_ratio, random_state=42)
    val_idx, test_idx = train_test_split(temp_idx, test_size=test_val_split, random_state=42)
    
    # Extending the lists to store indices for each set
    train_indices.extend(train_idx)
    val_indices.extend(val_idx)
    test_indices.extend(test_idx)

# Creating Subset objects using the selected indices for each set
train_set = Subset(train_dataset, train_indices)
val_set = Subset(train_dataset, val_indices)
test_set = Subset(train_dataset, test_indices)

# Transform functions
transform_train = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),      
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]), # mean and std for ImageNet dataset
])

transform_eval = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]), # mean and std for ImageNet dataset
])

train_set.dataset.transform = transform_train
val_set.dataset.transform = transform_eval
test_set.dataset.transform = transform_eval

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=num_workers)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=num_workers)

# Printing the sizes of the resulting sets
print(f"Number of samples in the training set: {len(train_set)}")
print(f"Number of samples in the validation set: {len(val_set)}")
print(f"Number of samples in the test set: {len(test_set)}")

Number of samples in the training set: 26880
Number of samples in the validation set: 5760
Number of samples in the test set: 5760


In [8]:
# Define the used model
model = model_resnet18
model.name = "resnet18"     # Set name for saving

model.fc = torch.nn.Linear(model.fc.in_features, num_classes)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
#optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

In [55]:
def save_model(trained_model):
    # Create Models Folder
    if not os.path.exists('./models'):
        print('Creating models directory')
        os.mkdir('./models')

    # Save the model
    if not os.path.exists(f'./models/{trained_model.name}.pth'):
        print(f'Saving Model {trained_model.name}')
        torch.save(trained_model, f'./models/{trained_model.name}.pth')

def load_model(model_name):
    if not os.path.exists(f'./models/{model_name}.pth'):
        print(f'Cannot Load {model_name}')
        return None
    print(f'Loading Model {model_name}')
    model = torch.load(f'./models/{model_name}.pth')
    model.eval()
    return model

In [10]:
def evaluate(model, data_loader, device):

    model.eval() 
    # valid_running_loss = 0.0
    valid_running_correct = 0
    counter = 0
    num_images = 0.0 # used to accumulate number of images
    
    #with torch.no_grad():
    for i, data in enumerate(data_loader, 0):
        counter += 1
        
        image, labels = data
        image = image.to(device)
        labels = labels.to(device)
        # Forward pass.
        outputs = model(image)
        # Calculate the loss.
        # loss = criterion(outputs, labels)
        # valid_running_loss += loss.item()
        # Calculate the accuracy.
        #preds = outputs.argmax(dim=1)
        preds = torch.max(outputs, 1).indices
        
        #valid_running_correct += preds.eq(labels).sum()
        valid_running_correct += torch.sum(preds == labels.data)
        num_images += len(labels)
        
    # Loss and accuracy for the complete epoch.
    # epoch_loss = valid_running_loss / counter
    lg.debug(f"valid_running_correct: {valid_running_correct.item()}")

    epoch_acc = valid_running_correct / num_images
    return epoch_acc

In [11]:
def train(model, trainLoader, valLoader, criterion, optimizer, num_epochs, device):
    model.to(device)
    train_loss = []
    train_acc = []

    # Start the training.
    if torch.cuda.is_available():
        print("Using CUDA")

    print('Training')
    
    lg.debug(f" len(trainLoader.dataset): {len(trainLoader.dataset)}")
    lg.debug(f" len(valLoader.dataset): {len(valLoader.dataset)}")

    
    for epoch in range(num_epochs):
        model.train()
        train_running_loss = 0.0
        train_running_correct = 0
        counter = 0
        num_images = 0.0 # used to accumulate number of images
        for i, data in enumerate(trainLoader,0):
            counter += 1
            image, labels = data
            image = image.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            # Forward pass.
            outputs = model(image)
            # Calculate the loss.
            loss = criterion(outputs, labels)
            train_running_loss += loss.item()
            # Calculate the accuracy.
            _, preds = torch.max(outputs.data, 1)
            train_running_correct += torch.sum(preds == labels.data)
            # Backpropagation
            loss.backward()
            # Update the weights.
            optimizer.step()    # Output training 
            num_images += len(image)
            if counter % 100 == 0:
                print(f'Epoch: {epoch+1}/{num_epochs}, Step: {counter}/{len(trainLoader)}, Loss: {loss.item():.4f}')


        # Validation
        valid_epoch_acc = evaluate(model, valLoader, device)
        lg.debug(f"train_running_loss: {train_running_loss}, train_running_correct: {train_running_correct}")
        
        # train_epoch_loss = train_running_loss / counter
        # train_loss.append(train_epoch_loss)
        # valid_loss.append(valid_epoch_loss)
        train_epoch_acc = train_running_correct  / num_images
        # train_acc.append(train_epoch_acc )

        print('Epoch: %d/%d, Training accuracy: %f, Loss: %f, Validation accuracy: %f' % (epoch+1, num_epochs, train_epoch_acc, loss.item(), valid_epoch_acc))

        # valid_acc.append(valid_epoch_acc)
        # lg.debug(f"Training loss mean: {train_epoch_loss :.3f}")
        # print(f"Validation acc: {valid_epoch_acc:.3f}")
        print('-'*50)


    return model

In [ ]:
# pretrained_model = train(model, dataloader_train, dataloader_validation, criterion, optimizer, num_epochs, device)

In [12]:
pretrained_model = train(model, train_loader, val_loader, criterion, optimizer, num_epochs, device)

DEBUG:root: len(trainLoader.dataset): 26880
DEBUG:root: len(valLoader.dataset): 5760


Using CUDA
Training
Epoch: 1/10, Step: 100/210, Loss: 2.7449
Epoch: 1/10, Step: 200/210, Loss: 1.6270


DEBUG:root:valid_running_correct: 4187
DEBUG:root:train_running_loss: 585.4522787332535, train_running_correct: 12009


Epoch: 1/10, Training accuracy: 0.446763, Loss: 1.592140, Validation accuracy: 0.726910
--------------------------------------------------
Epoch: 2/10, Step: 100/210, Loss: 1.1672
Epoch: 2/10, Step: 200/210, Loss: 1.0501


DEBUG:root:valid_running_correct: 4588
DEBUG:root:train_running_loss: 243.0085175037384, train_running_correct: 20840


...

Epoch: 8/10, Training accuracy: 0.936272, Loss: 0.225380, Validation accuracy: 0.857639
--------------------------------------------------
Epoch: 9/10, Step: 100/210, Loss: 0.2248
Epoch: 9/10, Step: 200/210, Loss: 0.2873


DEBUG:root:valid_running_correct: 4939
DEBUG:root:train_running_loss: 53.526989713311195, train_running_correct: 25546


Epoch: 9/10, Training accuracy: 0.950372, Loss: 0.207097, Validation accuracy: 0.857465
--------------------------------------------------
Epoch: 10/10, Step: 100/210, Loss: 0.1863
Epoch: 10/10, Step: 200/210, Loss: 0.2096


DEBUG:root:valid_running_correct: 4951
DEBUG:root:train_running_loss: 45.61228997260332, train_running_correct: 25859


Epoch: 10/10, Training accuracy: 0.962016, Loss: 0.193105, Validation accuracy: 0.859549
--------------------------------------------------


In [59]:
save_model(pretrained_model)

In [60]:
# validate_loss, validate_acc = evaluate(trained_model, dataloader_validation, criterion, device)
# print(f"Validation loss: {validate_loss:.3f}, validation acc: {validate_acc:.3f}")
test_acc = evaluate(pretrained_model, test_loader, device)
print(f"test acc: {test_acc:.3f}")


DEBUG:root:valid_running_correct: 4970


test acc: 0.863


In [61]:
def choose_100_eurosat():
    '''
    The `choose_100_eurosat()` function selects 100 images from the EuroSAT dataset and copies them to the `data/eurosat_validation` folder. It also creates a `data/eurosat_training` folder if it doesn't already exist.
    '''
    if not os.path.exists('./data/eurosat_validation'):
        print('Creating data directory')
        os.mkdir('./data/eurosat_validation')
    else:
        # Folder full of images -> delete all files in the directory
        shutil.rmtree(os.path.join('./data/eurosat_validation'))
        os.mkdir('./data/eurosat_validation')

    if not os.path.exists('./data/eurosat_training'):
        print('Creating training directory')
        os.mkdir('./data/eurosat_training')
    else:
        # Folder full of images -> delete all files in the directory
        shutil.rmtree(os.path.join('./data/eurosat_training'))
        os.mkdir('./data/eurosat_training')

    # Load the EuroSAT Categories
    eurosat_categories = [name for name in os.listdir('./data/eurosat/EuroSAT_RGB/') if os.path.isdir(os.path.join('./data/eurosat/EuroSAT_RGB/', name))]
    # Randomly select 5 categories
    lg.debug(f"Selecting 5 categories from {eurosat_categories} categories")
    selected_categories = np.random.choice(eurosat_categories, 5, replace=False)
    lg.debug(f"Selected categories: {selected_categories}")

    selected_images = []
    training_images = []

    # From each directory, randomly select 20 images
    for category in selected_categories:
        images = os.listdir(os.path.join('./data/eurosat/EuroSAT_RGB/', category))
        selected = random.sample(images, 20)
        selected_images.extend([(category, image) for image in selected])
        lg.debug(f"Selected {selected} from {category}")
        # Copy selected images to the selected directory
        for image in selected:
            if not os.path.exists(f"./data/eurosat_validation/{category}"):
                lg.debug(f"Creating {category} directory")
                os.mkdir(f"./data/eurosat_validation/{category}")
            shutil.copyfile(os.path.join('./data/eurosat/EuroSAT_RGB', category, image), os.path.join(f"./data/eurosat_validation/{category}", image))

    # From these 100 images, randomly select 5 images from each category for the training set
    for category in selected_categories:
        category_images = [image for (cat, image) in selected_images if cat == category]
        training = random.sample(category_images, 5)
        training_images.extend(training)
        lg.debug(f"Selected {training} for training")

        # Copy training images to the training directory
        for image in training:
            if not os.path.exists(f"./data/eurosat_training/{category}"):
                lg.debug(f"Creating {category} directory")
                os.mkdir(f"./data/eurosat_training/{category}")
            shutil.move(os.path.join(f"./data/eurosat_validation/{category}", image), os.path.join(f"./data/eurosat_training/{category}", image))

In [62]:
# Choose 100 images from EuroSAT dataset
choose_100_eurosat()


eurosat_train_dataset = torchvision.datasets.ImageFolder(
    root='./data/eurosat_training',
    transform=transforms.Compose([
        transforms.RandomResizedCrop(image_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]), # mean and std for ImageNet dataset
    ])
)

eurosat_validation_dataset = torchvision.datasets.ImageFolder(
    root='./data/eurosat_validation',
    transform=transforms.Compose([
        #transforms.Resize(256),
        #transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])
)

eurosat_dataloader_train = DataLoader(eurosat_train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
eurosat_dataloader_validation = DataLoader(eurosat_validation_dataset, batch_size=batch_size, shuffle = False, num_workers=num_workers)

DEBUG:root:Selecting 5 categories from ['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake'] categories
DEBUG:root:Selected categories: ['River' 'Industrial' 'Forest' 'AnnualCrop' 'Residential']
DEBUG:root:Selected ['River_999.jpg', 'River_1856.jpg', 'River_2178.jpg', 'River_171.jpg', 'River_645.jpg', 'River_9.jpg', 'River_1663.jpg', 'River_545.jpg', 'River_2462.jpg', 'River_950.jpg', 'River_505.jpg', 'River_1171.jpg', 'River_756.jpg', 'River_433.jpg', 'River_1945.jpg', 'River_442.jpg', 'River_239.jpg', 'River_1775.jpg', 'River_1290.jpg', 'River_1509.jpg'] from River
DEBUG:root:Creating River directory
DEBUG:root:Selected ['Industrial_1179.jpg', 'Industrial_1968.jpg', 'Industrial_1062.jpg', 'Industrial_643.jpg', 'Industrial_579.jpg', 'Industrial_365.jpg', 'Industrial_1413.jpg', 'Industrial_339.jpg', 'Industrial_1068.jpg', 'Industrial_1027.jpg', 'Industrial_2488.jpg', 'Industrial_2164.jpg', 'Industrial_21

In [65]:
# Load the pretrained model
model_resnet18_fine = load_model("resnet18")

# Define loss function and optimizer for fine-tuning
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

num_epochs = 10

eurosat_trained_model = train(model_resnet18_fine, eurosat_dataloader_train, eurosat_dataloader_validation, criterion, optimizer, num_epochs, device)


DEBUG:root: len(trainLoader.dataset): 25
DEBUG:root: len(valLoader.dataset): 75


Loading Model resnet18
Using CUDA
Training


DEBUG:root:valid_running_correct: 1
DEBUG:root:train_running_loss: 6.916092395782471, train_running_correct: 0


Epoch: 1/10, Training accuracy: 0.000000, Loss: 6.916092, Validation accuracy: 0.013333
--------------------------------------------------


DEBUG:root:valid_running_correct: 1
DEBUG:root:train_running_loss: 6.793035984039307, train_running_correct: 0


...

Epoch: 8/10, Training accuracy: 0.040000, Loss: 6.510320, Validation accuracy: 0.000000
--------------------------------------------------


DEBUG:root:valid_running_correct: 0
DEBUG:root:train_running_loss: 6.671468734741211, train_running_correct: 1


Epoch: 9/10, Training accuracy: 0.040000, Loss: 6.671469, Validation accuracy: 0.000000
--------------------------------------------------


DEBUG:root:valid_running_correct: 0
DEBUG:root:train_running_loss: 6.399109363555908, train_running_correct: 1


Epoch: 10/10, Training accuracy: 0.040000, Loss: 6.399109, Validation accuracy: 0.000000
--------------------------------------------------
